In [1]:
! which python3

/bin/python3


In [2]:
! /bin/pip install -r ../requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 40.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 KB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 KB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 KB 55.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 KB 27.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 KB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 56.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 KB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 46.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import sys 
sys.path.append('/workspaces/AI_Chatbot')

In [4]:
import logging
import os

from langchain_community.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata

from constants import SOURCE_DIRECTORY, CHROMA_SETTINGS, PERSIST_DIRECTORY
from langchain.document_loaders import PDFPlumberLoader, DirectoryLoader

from semantic_chunking_helper import SematicChunkingHelper

import torch

In [5]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(filename)s:%(lineno)s - %(message)s",
    level=logging.INFO,
)

source_dir = '/workspaces/AI_Chatbot/.DOCS.t/'

model_name='dangvantuan/vietnamese-embedding'
persist_dir=PERSIST_DIRECTORY + "_" + model_name,
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(model_name, device=device)
test_sentence = ["Xin chào, đây là thử nghiệm"]
try:
    embedding_output = model.encode(test_sentence)
    print(f"Embedding output: {embedding_output}")
except RuntimeError as e:
    print(f"Error during embedding: {e}")


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
2024-10-02 08:28:53,484 - INFO - SentenceTransformer.py:107 - Load pretrained SentenceTransformer: dangvantuan/vietnamese-embedding
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning

Embedding output: [[ 6.46414682e-02 -2.79253796e-02  1.80540472e-01 -1.94376945e-01
  -1.48897991e-01  2.75223851e-02  2.05813363e-01 -5.98897338e-01
  -4.46321845e-01 -1.76655903e-01  3.60757351e-01  4.81040299e-01
   2.27497533e-01  4.06815171e-01 -4.94004041e-02  4.34967816e-01
   5.55163383e-01  3.12171012e-01 -3.57532114e-01 -4.05472875e-01
   6.68190300e-01 -1.50099978e-01  1.32202199e-02  5.47649622e-01
  -1.10528517e+00 -1.68719545e-01 -3.90341312e-01 -4.53529835e-01
   4.44119096e-01  5.26073538e-02 -3.65168065e-01 -6.98505715e-02
  -8.66786689e-02  2.25236908e-01  4.20317203e-01 -7.87826002e-01
  -2.32527956e-01  8.07268560e-01  8.83637369e-02 -1.72254398e-01
   1.68198735e-01 -1.41508698e-01 -5.95090762e-02 -7.30464682e-02
  -2.53133848e-02  7.49091804e-02 -2.63811201e-01  2.31989726e-01
   9.39007755e-03  1.70397371e-01 -2.56641418e-01  5.94984442e-02
  -2.43142381e-01  5.56152165e-01 -1.36461377e-01  6.24050163e-02
   5.56508154e-02 -4.96622585e-02  6.10720694e-01 -8.39585

In [7]:
! export CUDA_LAUNCH_BLOCKING=1

# PyPDFDirectoryLoader, supports loading dpf files
docs = DirectoryLoader(
    source_dir, glob="**/*.pdf", loader_cls=PDFPlumberLoader
).load()
chunks = filter_complex_metadata(docs)

# model_name = "dangvantuan/vietnamese-embedding"
embeddings = HuggingFaceInstructEmbeddings(
    model_name=model_name,
    model_kwargs={"device": device},  # note for Apple Silicon Chip use "mps"
)
semantic_chunking = SematicChunkingHelper(
    docs=chunks, embeddings=embeddings, buffer_size=2, breakpoint_threshold=50
)
Chroma.from_texts(
    texts=semantic_chunking.text_chunks,
    embedding=embeddings,
    client_settings=CHROMA_SETTINGS,
    persist_directory=persist_dir,
)


2024-10-02 08:29:29,438 - INFO - SentenceTransformer.py:107 - Load pretrained SentenceTransformer: dangvantuan/vietnamese-embedding


TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'